In [ ]:
import os

In [ ]:
os.environ

In [ ]:
migs= 'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'

In [ ]:
env = dict(os.environ)

In [ ]:
env['CUDA_VISIBLE_DEVICES']=migs

In [ ]:
os.environ = env

In [ ]:
os.environ

In [1]:
import os
import ray

# Set CUDA_VISIBLE_DEVICES to control which GPUs are visible to Ray
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6"#'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'#"0,1,2,3,4,5,6"

# Initialize Ray with the appropriate configuration
ray.init()


2023-02-24 19:19:58,723	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.16
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


In [2]:
resources = ray.cluster_resources()
print(resources)


{'object_store_memory': 1053027532.0, 'CPU': 20.0, 'GPU': 1.0, 'node:127.0.0.1': 1.0, 'memory': 2106055067.0}


In [3]:
import numpy as np
import gym
from gym_CTgraph import CTgraph_env
from gym_CTgraph.CTgraph_plot import CTgraph_plot
from gym_CTgraph.CTgraph_conf import CTgraph_conf
from gym_CTgraph.CTgraph_images import CTgraph_images
import json
import argparse
#import json
import random
import matplotlib.pyplot as plt
import timeit

from gym.spaces import Box, Discrete, Dict
from ray.tune.registry import register_env
from ray import tune

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import argparse
import gym
from gym.spaces import Discrete, Box
import numpy as np
#import os
import random

import ray
from ray import tune
from ray.rllib.agents import ppo
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.logger import pretty_print

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [4]:
'''import ray
ray.init()
resources = ray.cluster_resources()
print(resources)'''


'import ray\nray.init()\nresources = ray.cluster_resources()\nprint(resources)'

In [5]:
from ray import tune

In [6]:
def env_creator(env_config={}):
    # fetch the parameters from the json file
    configuration = CTgraph_conf("/home/lunet/cocp5/Documents/rllib20/graph09.json")
    conf_data = configuration.getParameters()
    # print configration data
    print(json.dumps(conf_data, indent=3))
    env = gym.make('CTgraph-v0')
    # instantiate the maze
    #start = timeit.timeit()
    #env = gym.make('CTgraph-v0')
    #end = timeit.timeit()
    #print(end - start)

    imageDataset = CTgraph_images(conf_data)
    observation = env.init(conf_data, imageDataset)

    # get a random path from the maze
    #high_reward_path = np.array(np.random.randint(1, conf_data['graph_shape']['branching_factor'], size=conf_data['graph_shape']['depth']))
    # use this random path to set the path to the high reward. Note that the maze would have already a high_reward_path from the initialisation
    #env.set_high_reward_path(high_reward_path)
    #seed_everything(env)  
    
    return env

In [7]:
register_env("CT-Graph_d2_b2_wp05_crv0_MDP", env_creator)

In [8]:
tune.run("DDPPO",
             # algorithm specific configuration
             config={"env": "CT-Graph_d2_b2_wp05_crv0_MDP",  #
                     "framework": "torch",
                     "seed":958,
                     #"num_gpus": 7,"num_rollout_workers": 1,
                     "num_envs_per_worker": 1,
                     "num_gpus_per_worker": 0,
                     "train_batch_size": 128,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 64,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.0015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2
                     },
             local_dir="CT-Graph_d2_b2_wp05_crv0_MDP04",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"timesteps_total": 1024000}
             )


2023-02-24 19:20:18,953	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=10052) WARNING:tensorflow:From C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\tensorflow\python\util\deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=10052) Instructions for updating:
(pid=10052) experimental_relax_shapes is deprecated, use reduce_retracing instead
(DDPPO pid=10052) 2023-02-24 19:20:32,601	WARNING algorithm_config.py:596 -- Cannot create DDPPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(DDPPO pid=10052) 2023-02-24 19:20:32,602	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num

(RolloutWorker pid=33556) ---------------------------------------------------
(RolloutWorker pid=33556)              The CT-graph environments             
(RolloutWorker pid=33556) ---------------------------------------------------
(RolloutWorker pid=33556) Reading configuration parameters from  /home/lunet/cocp5/Documents/rllib20/graph09.json


(RolloutWorker pid=33556) 2023-02-24 19:20:40,011	ERROR worker.py:772 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=33556, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000184C8AC7460>)
(RolloutWorker pid=33556)   File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
(RolloutWorker pid=33556)   File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
(RolloutWorker pid=33556)   File "python\ray\_raylet.pyx", line 803, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=33556)   File "C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\ray\_private\function_manager.py", line 674, in actor_method_executor
(RolloutWorker pid=33556)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=33556)   File "C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in 

Trial name,trial_id
DDPPO_CT-Graph_d2_b2_wp05_crv0_MDP_466ff_00000,466ff_00000


(DDPPO pid=10052) 2023-02-24 19:20:40,065	ERROR actor_manager.py:496 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=11528, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000224F4497430>)
(DDPPO pid=10052)   File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
(DDPPO pid=10052)   File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
(DDPPO pid=10052)   File "python\ray\_raylet.pyx", line 803, in ray._raylet.execute_task.function_executor
(DDPPO pid=10052)   File "C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\ray\_private\function_manager.py", line 674, in actor_method_executor
(DDPPO pid=10052)     return method(__ray_actor, *args, **kwargs)
(DDPPO pid=10052)   File "C:\Users\chper\anaconda3\envs\rllib20c\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in _resume_span
(DDPPO pid=10052)     return 

(RolloutWorker pid=11528) ---------------------------------------------------
(RolloutWorker pid=11528)              The CT-graph environments             
(RolloutWorker pid=11528) ---------------------------------------------------
(RolloutWorker pid=11528) Reading configuration parameters from  /home/lunet/cocp5/Documents/rllib20/graph09.json


TuneError: ('Trials did not complete', [DDPPO_CT-Graph_d2_b2_wp05_crv0_MDP_466ff_00000])